In [1]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import chain
from datetime import timedelta, date, datetime, timezone
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

from secrets import *

In [2]:
dbname = 'cluj'
engine = create_engine('postgres://%s:%s@localhost/%s'%('docker','docker',dbname))

if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [16]:
sql = """
    SELECT DISTINCT name as boxscore_name FROM boxscores
"""
boxscore_names = pd.read_sql(sql, engine)
boxscore_names.shape

(638, 1)

In [17]:
boxscore_names.head()

,boxscore_name
0,Antonio Blakeney
1,Langston Galloway
2,Quincy Acy
3,Karl-Anthony Towns
4,Will Barton


In [18]:
league_id = 84057

In [19]:
url = 'https://fantasy.espn.com/apis/v3/games/FBA/seasons/2020/segments/0/leagues/' + str(league_id)
params={"view": "kona_player_info"}
cookies = {"swid": AUTH['swid'],
           "espn_s2": AUTH['espn_s2']}
r = requests.get(url, params=params, cookies=cookies)
data = r.json()
data.keys()

dict_keys(['players'])

In [20]:
espn_names = []
for player in data['players']:
    for stat_item in player['player']['stats']:
        if stat_item['id'] == '002020':
            espn_names.append(player['player']['fullName'])
espn_names_df = pd.DataFrame({'espn_name':espn_names})
espn_names_df.head()

,espn_name
0,Metta World Peace
1,Vince Carter
2,Jamal Crawford
3,Kevin Garnett
4,Manu Ginobili


In [23]:
merged = espn_names_df.merge(boxscore_names, how='outer', left_on='espn_name', right_on='boxscore_name')

In [28]:
merged.loc[merged.espn_name.isnull()].to_csv('espn_names.csv')

In [27]:
merged.loc[merged.boxscore_name.isnull()].shape

(418, 2)

In [117]:
espn_names_df.loc[espn_names_df.espn_name.str.contains('Brown')]

,espn_name
160,Bobby Brown
403,Alec Brown
409,Lorenzo Brown
417,Markel Brown
431,Anthony Brown
466,Jabari Brown
598,Sterling Brown
796,Jaylen Brown
849,Bruce Brown
924,Troy Brown Jr.
